### Decorators Application (Logger, Stacked Decorators)
In this example we're going to create a utility decorator that will log function calls (to the console, but in practice you would be writing your logs to a file (e.g. using Python's built-in logger), or to a database, etc.)

In [7]:
def logged(fn):
    from functools import wraps
    from datetime import datetime,timezone
    
    @wraps(fn)
    def inner(*args,**kwargs):
        run_dt = datetime.now(timezone.utc)
        result = fn(*args,**kwargs)
        print("{0}: called at - {1}".format(fn.__name__, run_dt))
        return result
    return inner

In [8]:
@logged
def fun_1():
    pass

In [9]:
fun_1()

fun_1: called at - 2023-11-15 14:03:49.186762+00:00


In [10]:
def timed(fn):
    from functools import wraps
    from time import perf_counter
    
    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        print('{0} ran for {1:.6f}s'.format(fn.__name__, end-start))
        return result
    
    return inner

In [11]:
@timed
@logged
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul,range(1,n+1))

In [12]:
factorial(5)

factorial: called at - 2023-11-15 14:09:42.960957+00:00
factorial ran for 0.000095s


120

Note that the order in which we stack the decorators can make a difference!

Remember that this is because our stacked decorators essentially amounted to:

In [13]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = timed(logged(factorial))

In [14]:
factorial(5)

factorial: called at - 2023-11-15 14:11:23.693257+00:00
factorial ran for 0.000060s


120


So in this case the `timed` decorator will be called first, followed by the `logged` decorator.

You may wonder why the printed output seems reversed. Look at how the decorators were defined - they first ran the function passed in, and **then** printed the result.

So in the above example, a simplified look at what happens in each decorator:

* `timed(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints timing
    
    
* `logged(fn)(*args, **kwargs)`:
    1. calls `fn(*args, **kwargs)`
    2. prints log info

So, calling
`factorial = timed(logged(factorial))`

is equivalent to:

<pre>
fn = logged(factorial)
factorial = timed(fn)

factorial(n) --> call timed(fn)(n)
             --> call fn(n), then print timing
             --> call logged(original_factorial)(n), then print timing
             --> call original_factorial(n), then log, then print timing
</pre>

So as you can see, the `timed` decorator ran first, but it called the logged decorated function first, then printed the result - hence why the print output seems reversed.

But in the following case, the `logged` decorator will run first, followed by the `timed` decorator:

In [15]:
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

factorial = logged(timed(factorial))

In [16]:
factorial(5)

factorial ran for 0.000020s
factorial: called at - 2023-11-15 14:14:47.932917+00:00


120

Or, using the **@** notation:

In [18]:
@logged
@timed
def factorial(n):
    from operator import mul
    from functools import reduce
    
    return reduce(mul, range(1, n+1))

In [19]:
factorial(5)

factorial ran for 0.000030s
factorial: called at - 2023-11-15 14:15:54.817963+00:00


120

---
You may wonder whether this really matters in practice. And yes, it can.

Consider an API that contains various functions that can be called. However, endpoints are secured and can only be run by authenticated users who have some specific role(s). If they do not have the role you want to return an unauthorized error. But if they do, then you want to log that they called the endpoint.

In this case you may have one decorator that is used to check authentication and permissions (and immediately return an unauthorized error from the API if applicable), and the other to log the call. 

If you decorated it this way:

<pre>
@log
@authorize
def my_endpoint():
    pass
</pre>

then the call would always be logged.

But, in this instance:

<pre>
@authorize
@log
def my_endpoint():
    pass
</pre>

your endpoint would only get logged if the user passed the `authorize` test.